In [1]:
import pandas as pd
import numpy as np
import pyodbc
import json
import string
import pickle
from jieba import lcut
from collections import Counter
from statistics import mean
from scipy.sparse import csr_matrix
from tqdm import tqdm
import time
import pkuseg
import thulac  
from gensim import corpora

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold # import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

from sentence_transformers import models, SentenceTransformer
import numpy as np
import string
pd.set_option('display.max_rows', 50000)

random_seed = 751031
np.random.seed(random_seed)

In [2]:
#function zone
pku_seg = pkuseg.pkuseg()    
thu_lac = thulac.thulac(seg_only=True)
def Bulid_dict(message_list):
# 總分詞
    ls_of_words = [pku_seg.cut(text) for text in message_list]
    #ls_of_words = [thu_lac.cut(i, text=True) for i in message_list]
    rs = []
    for j in tqdm(range(len(ls_of_words))):
        temp = []
        for k in range(len(ls_of_words[j])):
            temp.append(remove_punctuation(ls_of_words[j][k]))
        rs.append(temp)    
    
    # 構造詞典
    dictionary = corpora.Dictionary(rs)
    dt = dictionary.token2id
    length = len(dt)
    return dt, length

def vectorize(ls_of_words, dt):
    length = len(dt)
    # 句向量
    ls_of_wid = []
    for words in ls_of_words:#ls_of_words 
        vector = [0] * length
        for word in words:
            try:
                vector[dt[word]] += 1
            except:
                continue
        ls_of_wid.append(vector) 
    return ls_of_wid

def add_feature(sr):
    num_only = int(sr.isdigit())
    abc_only = int(sr.encode( 'UTF-8').isalpha())
    #numorabc_only = int(sr.isalnum())
    def is_contains_chinese(strs):
        for _char in strs:
            if '\u4e00' <= _char <= '\u9fa5':
                return 1
        return 0
 
    #检验是否全是中文字符
    def is_all_chinese(strs):
        for _char in strs:
            if not '\u4e00' <= _char <= '\u9fa5':
                return 0
        return 1
    #allchinese = is_all_chinese(sr)
    #containchinese = is_all_chinese(sr)   
    return [num_only, abc_only]


def remove_punctuation(s): 
    punctuation = '''''。：，。？表情\n!！()-[]{};:'"\,<>./[email protected]#$%^&*_~ '''+ string.digits+string.whitespace+ string.punctuation
    #'''''\n!()-[]{};:'"\,<>./[email protected]#$%^&*_~！（）－［］｛｝；：’＂＼，＜＞．／＃＄％︿＆＊＿～'''
    my_str = s 
    no_punct = "" 
    for char in my_str: 
        if char not in punctuation: 
            no_punct += char 
    return(no_punct) 

def change_string(articut, text):
    if text == '':
        tmp = ''
    else:
        result = articut.parse(text)
        content = articut.getContentWordLIST(result)
        tmp = ''
        for sentense in content:
            for word in sentense:
                tmp += word[-1]
    return tmp

Model loaded succeed


## 建構一個總辭典

In [3]:
data_modify = pd.read_excel("C://Users//user//Desktop//project//up//input//input_v5.xlsx", index_col = None, engine='openpyxl', header=0)
data_modify = data_modify[['message', '正确意图']]
data_modify = data_modify[~data_modify.loc[:, '正确意图'].isna()]
data_modify.columns = ['message', 'label']

In [4]:
change_dict = {'到帐问题':0, '提款卖币问题':1, '存款买币问题':2, '银行卡问题':3, '操作问题':4, '订单取消问题':5, '非业务意图':6}
change_dict_verse = {v: k for k, v in change_dict.items()}
data_modify = data_modify[data_modify.label.isin(list(change_dict.keys()))].reset_index(drop=True)
print(data_modify.shape)
#data_modify.loc[:, 'message'] = data_modify.loc[:, 'message'].apply(remove_punctuation)
data_modify.loc[:, 'message'] = [i.upper()  for i in data_modify.message]

(4675, 2)


In [5]:
# -*- coding: utf-8 -*-
model_name = 'uer/roberta-base-finetuned-chinanews-chinese'#'imxly/sentence_roberta_wwm_ext'#'imxly/sentence_roberta_wwm_ext'#
#model_name ='sentence-transformers/paraphrase-xlm-r-multilingual-v1'

word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
sbert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [6]:
def embedding(t, model=sbert_model):
    result = model.encode(t)
    result = result / np.linalg.norm(result)
    return result

In [7]:
'''
train_bert = np.zeros((0, 768), float)
for j in tqdm(data_modify.message):
    j = remove_punctuation(j)
    train_bert = np.append(train_bert, embedding(j).reshape(1,768), axis=0)
    
np.save('train_bert.npy', train_bert)
'''

"\ntrain_bert = np.zeros((0, 768), float)\nfor j in tqdm(data_modify.message):\n    j = remove_punctuation(j)\n    train_bert = np.append(train_bert, embedding(j).reshape(1,768), axis=0)\n    \nnp.save('train_bert.npy', train_bert)\n"

In [8]:
train_bert = np.load('train_bert.npy')

In [9]:
dt, length = Bulid_dict(data_modify.message)
'''
#save json and load json file
with open('C://Users//user//Desktop//aicode-master//textLearn//data//up_dict.json', 'w') as fp:
    json.dump(dt, fp)

with open('C://Users//user//Desktop//aicode-master//textLearn//data//up_dict.json') as json_file: 
    dt = json.load(json_file) 
'''

100%|██████████████████████████████████████████████████████████████████████████| 4675/4675 [00:00<00:00, 361451.29it/s]


"\n#save json and load json file\nwith open('C://Users//user//Desktop//aicode-master//textLearn//data//up_dict.json', 'w') as fp:\n    json.dump(dt, fp)\n\nwith open('C://Users//user//Desktop//aicode-master//textLearn//data//up_dict.json') as json_file: \n    dt = json.load(json_file) \n"

In [10]:
'''
def is_contains_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False
look = []
for i in list(dt.keys()):
    if ('US' not in i) & (~is_contains_chinese(i)):
        look.append(i)
        del dt[i]
        
x = 0
for i in list(dt.keys()):
    dt[i] = x
    x += 1
'''

"\ndef is_contains_chinese(strs):\n    for _char in strs:\n        if '一' <= _char <= '龥':\n            return True\n    return False\nlook = []\nfor i in list(dt.keys()):\n    if ('US' not in i) & (~is_contains_chinese(i)):\n        look.append(i)\n        del dt[i]\n        \nx = 0\nfor i in list(dt.keys()):\n    dt[i] = x\n    x += 1\n"

# 预处理

In [11]:
CountVector = CountVectorizer()
CountVectorizer_fit = CountVector.fit_transform(data_modify.message)
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)
tv.fit(data_modify.message)
tv_fit = tv.transform(data_modify.message)
#pickle.dump(tv, open('tv.pkl', 'wb'))

In [12]:
train_list = [pku_seg.cut(text) for text in data_modify.message]#[lcut(text) for text in data_modify.message]
train_list_vec = vectorize(train_list, dt)

In [13]:
data_modify['char_count'] = data_modify['message'].apply(len)
data_modify['word_count'] = data_modify['message'].apply(lambda x: len(x.split()))
data_modify['word_density'] = data_modify['char_count'] / (data_modify['word_count']+1)
data_modify['punctuation_count'] = data_modify['message'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))

add_feature_ = data_modify[['word_count', 'word_density', 'punctuation_count']].values
add_feature_isin = np.array([add_feature(i) for i in data_modify.message])

In [14]:
train_data = np.c_[np.array(train_list_vec), 
                   add_feature_isin, 
                   tv_fit.toarray(),
                   add_feature_]
maxminscaler = MinMaxScaler()
maxminscaler.fit(train_data)
train_data = maxminscaler.transform(train_data)
#train_data = np.c_[train_data, 
#                   train_bert]
#
label_change = [change_dict[i] for i in data_modify.label]
data_modify.loc[:, 'label'] = label_change

In [15]:
#pickle.dump(maxminscaler, open('maxminscaler.pkl', 'wb'))

In [16]:
# 資料切分
train_y, test_y, train_x, test_x = train_test_split(data_modify.label, 
                                                    train_data,#train_list_vec, 
                                                    test_size=0.1, 
                                                    random_state=random_seed)
train_y = train_y.reset_index(drop=True)
del(train_data)

In [17]:
train_x_oversameple = np.zeros((0,train_x.shape[1]), float)
train_y_oversameple = np.zeros((0,), float)

max_sample = train_y.shape[0]#Counter(train_y).most_common(1)[0][1]
np.random.seed(random_seed)
for i in train_y.unique():
    ta = list(np.random.choice(list(train_y[train_y==i].index), max_sample))
    train_y_oversameple = np.append(train_y_oversameple, train_y[ta], axis=0)
    train_x_oversameple = np.append(train_x_oversameple, train_x[ta], axis=0)

In [18]:
n_fold = 5
kf = KFold(n_splits=n_fold, random_state=random_seed, shuffle=True)
accuracy_list = []
for train_index, valid_index in kf.split(train_x):
    #print(valid_index)
    X_train, X_test = train_x[train_index], train_x[valid_index]
    y_train, y_test = train_y[train_index], train_y[valid_index]
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    max_sample = X_train.shape[0]#Counter(train_y).most_common(1)[0][1]
    X_train_oversameple = np.zeros((0, X_train.shape[1]), float)
    y_train_oversameple = np.zeros((0,), float)
    np.random.seed(random_seed)
    for i in range(7):
        ta = list(np.random.choice(list(y_train[y_train==i].index), max_sample))
        y_train_oversameple = np.append(y_train_oversameple, y_train[ta], axis=0)
        X_train_oversameple = np.append(X_train_oversameple, X_train[ta], axis=0)    
    classifier = MultinomialNB()  # 樸素貝葉斯分類器
    classifier.fit(X_train_oversameple, y_train_oversameple)
    # 模型測評
    score = classifier.score(X_test, y_test)
    accuracy_list.append(score)
    #del(X_train, X_test, y_train, y_test, classifier)
    
print('{k}_fold cross valid accuracy mean:{mean}, max:{max}, min:{min}'.format(k=n_fold, 
                                                                               mean=mean(accuracy_list),
                                                                               max=max(accuracy_list),
                                                                               min=min(accuracy_list)))

5_fold cross valid accuracy mean:0.6831444299146192, max:0.6983372921615202, min:0.6706302021403091


In [19]:
# 貝葉斯模型訓練
classifier = MultinomialNB()  # 樸素貝葉斯分類器
classifier.fit(train_x_oversameple, train_y_oversameple)

# 模型測評
score = classifier.score(test_x, test_y)
print('Test set Accuracy：'+ str(round(score*100, 2))+'%')

Test set Accuracy：70.73%


In [20]:
classifier.coef_.shape

(7, 6578)

In [21]:
'''
#save model and load the model
filename = 'C://Users//user//Desktop//aicode-master//textLearn//models//nb//nb.sav'
pickle.dump(classifier, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))
'''

"\n#save model and load the model\nfilename = 'C://Users//user//Desktop//aicode-master//textLearn//models//nb//nb.sav'\npickle.dump(classifier, open(filename, 'wb'))\n#classifier = pickle.load(open(filename, 'rb'))\n"

## Model -logistic regression

In [27]:
n_fold = 5
random_seed = 751031
kf = KFold(n_splits=n_fold, random_state=random_seed, shuffle=True)
accuracy_list = []
for train_index, valid_index in kf.split(train_x):
    X_train, X_test = train_x[train_index], train_x[valid_index]
    y_train, y_test = train_y[train_index], train_y[valid_index]
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    #for i in range(7):
        #x = list(y_test).count(i)
        #ratio = x/ y_test.shape[0]
        #print(change_dict_verse[i]+ str(round(ratio*100,2))+ '%')
    #print("----------------------------")
    max_sample = X_train.shape[0]#Counter(train_y).most_common(1)[0][1]
    X_train_oversameple = np.zeros((0, X_train.shape[1]), float)
    y_train_oversameple = np.zeros((0,), float)
    np.random.seed(random_seed)
    for i in range(7):
        ta = list(np.random.choice(list(y_train[y_train==i].index), max_sample))
        y_train_oversameple = np.append(y_train_oversameple, y_train[ta], axis=0)
        X_train_oversameple = np.append(X_train_oversameple, X_train[ta], axis=0)    
    lr = LogisticRegression(solver='saga',
                            multi_class='multinomial',
                            C=1,
                            penalty='l1',
                            fit_intercept=True,
                            max_iter=50000,
                            random_state=4180,
                            class_weight='balanced',
                            n_jobs=-1)
    lr.fit(csr_matrix((X_train_oversameple), dtype=float), 
           y_train_oversameple)
    # 模型測評
    score = lr.score(csr_matrix((X_test), dtype=float), 
                     y_test)
    accuracy_list.append(score)
    print('-----')    
print('{k}_fold cross valid accuracy mean:{mean}, max:{max}, min:{min}'.format(k=n_fold, 
                                                                               mean=mean(accuracy_list),
                                                                               max=max(accuracy_list),
                                                                               min=min(accuracy_list)))

-----
-----
-----
-----
-----
5_fold cross valid accuracy mean:0.8288549148310602, max:0.8408551068883611, min:0.8204518430439952


In [28]:
X_train_oversameple.shape

(23562, 6578)

In [24]:
s = time.time()
lr = LogisticRegression(solver='saga',
                        multi_class='multinomial',
                        C=1,
                        penalty='l1',
                        fit_intercept=True,
                        max_iter=50000,
                        random_state=4180,
                        class_weight='balanced',
                        n_jobs=-1)
lr.fit(csr_matrix((train_x_oversameple), dtype=float), train_y_oversameple)
accuracy = lr.score(csr_matrix((test_x), dtype=float), test_y)
print('LR - Test set Accuracy：'+str(round(accuracy*100, 2))+'%')
print(time.time()-s)

LR - Test set Accuracy：83.12%
90.62537479400635


In [25]:
PRED_Y = lr.predict(csr_matrix(test_x, dtype=float))

In [26]:
for i in range(7):
    x = list(test_y).count(i)
    ratio = x/ test_y.shape[0]
    print(str(x)+ '_'+ str(i)+  '_'+ str(round(ratio*100,2))+ '%')

58_0_12.39%
41_1_8.76%
64_2_13.68%
2_3_0.43%
76_4_16.24%
7_5_1.5%
220_6_47.01%


In [ ]:
for i in range(7):
    x = list(PRED_Y).count(i)
    ratio = x/ test_y.shape[0]
    print(str(x)+ '_'+ str(i)+  '_'+ str(round(ratio*100,2))+ '%')

In [ ]:
#save model and load the model
filename = 'C://Users//user//Desktop//LR_UP'
pickle.dump(lr, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))

In [ ]:
'''
#save model and load the model
filename = 'C://Users//user//Desktop//aicode-master//textLearn//models//nb//nb.sav'
pickle.dump(classifier, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))
'''

# pred result (单一)

In [ ]:
''''''
strs = '游戏没法充值'
test_1 = [lcut(strs)]
dict_key = list(dt.keys())
tmp = []
for i in test_1[0]:
    if i in dict_key:
        tmp.append(i)
test_1 = [tmp]


strs_df = pd.DataFrame({'message':[strs]})
strs_df['char_count'] = strs_df['message'].apply(len)
strs_df['word_count'] = strs_df['message'].apply(lambda x: len(x.split()))
strs_df['word_density'] = strs_df['char_count'] / (strs_df['word_count']+1)
strs_df['punctuation_count'] = strs_df['message'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))


add_feature_test = strs_df[['word_count', 'word_density', 'punctuation_count']].values
tran = tv.transform(pd.Series(strs))

#tfidf = tfidf_vect.transform(pd.Series(strs))
test_1 = vectorize(test_1, dt)
test_1 = np.c_[np.array(test_1), np.array(add_feature(strs)).reshape((1, 2)), tran.toarray(), add_feature_test]

maxminscaler = pickle.load(open('maxminscaler.pkl', 'rb'))
test_1 = maxminscaler.transform(test_1)
#test_1 = np.c_[np.array(test_1), np.array(add_feature(strs)).reshape((1, 2)), add_feature_test]

In [ ]:
y2 = lr.predict(test_1)
change_dict_verse[y2[0]]

In [ ]:
#test_1 = np.array(test_1)
y = classifier.predict(test_1)
change_dict_verse[y[0]]

In [ ]:
n_fold = 5
kf = KFold(n_splits=n_fold, random_state=1234, shuffle=True)
result_prob_list = np.zeros((test_1.shape[0], 7), dtype=float)
for train_index, valid_index in kf.split(train_x_oversameple):
    #print(valid_index)
    X_train, X_test = train_x_oversameple[train_index], train_x_oversameple[valid_index]
    y_train, y_test = train_y_oversameple[train_index], train_y_oversameple[valid_index]
    classifier = MultinomialNB()  # 樸素貝葉斯分類器
    classifier.fit(X_train, y_train)
    result_prob_list += classifier.predict_proba(test_1)
    del(X_train, X_test, y_train, y_test, classifier)
change_dict_verse[np.argmax(result_prob_list)]

In [ ]:
'''
for i in data_modify.label.unique():
    #print(i)
    x = list(data_modify.label).count(i)
    ratio = x/ data_modify.shape[0]
    print(change_dict_verse[i]+ str(round(ratio*100,2))+ '%')
'''

In [ ]:
data_modify.label.hist()

In [ ]:
data_modify.label.unique()

In [ ]:
change_dict_verse